In [1]:
import glob
import numpy as np 
import time
import math
import random
from scipy import linalg as LA
import pandas as pd
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.svm import NuSVC
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import Perceptron
import statsmodels.api as sm
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
from termcolor import colored
import matplotlib as mpl
import matplotlib.pyplot as plt
from scipy import optimize
from sklearn.svm import LinearSVC
from datetime import datetime
from scipy.stats import entropy
import os
import pickle
import trjtrypy as tt
from trjtrypy.distances import d_Q
from trjtrypy.distances import d_Q_pi
import trjtrypy.visualizations as vs
from scipy.spatial import distance
from trjtrypy.featureMappings import curve2vec

In [2]:
def ExpCurve2Vec(points,curves,mu):
    D = tt.distsbase.DistsBase()
    a = np.array([np.exp(-1*np.power(D.APntSetDistACrv(points,curve),2)/(mu)**2) for curve in curves])
    return a

In [3]:
I = glob.glob('Geolife Trajectories 1.3/**/*.plt', recursive=True)
S = list(set([I[i][:45] for i in range(len(I))]))
user_idx = np.array([S[i][30:33] for i in range(len(S))])
print(user_idx[:10])
len(S), S[0]

['079' '139' '053' '008' '153' '032' '036' '159' '063' '137']


(182, 'Geolife Trajectories 1.3/Data/079/Trajectory/')

In [4]:
def read_file(path):
    J = glob.glob(path, recursive=True)
    data = [0] * len(J)
    c = 0
    for j in range(len(J)):
        data[j] = []
        with open(J[j], "r") as f:
            for line in f:
                if c > 6:
                    item = line.strip().split(",")
                    if len(item) == 7:
                        data[j].append(np.array([float(item[0]), float(item[1]), 
                                                 float(item[4])]))
                c += 1
        data[j] = np.array(data[j])
    return np.array(data)

### data1[i] in the following is user_i from Beijing dataset 

trajectories have time dimension in data1

In [5]:
start_time = time.time()
user_idx_int = np.array(list(map(int, user_idx)))
data1 = [0] * len(S)
for i in range(len(S)):
    idx = np.where(user_idx_int == i)[0][0]
    path = S[idx]+'*.plt'
    J = glob.glob(path, recursive=True)
    data1[i] = read_file(path)
data1 = np.array(data1)
print(time.time() - start_time)

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  app.launch_new_instance()


106.70930314064026


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:9: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  if __name__ == '__main__':


In [6]:
data_1 = data1 + 0
len(data_1)

182

In [7]:
def remove_segments(traj): # removes stationary points
    p2 = traj[:,:2][1:]
    p1 = traj[:,:2][:-1]
    L = ((p2-p1)*(p2-p1)).sum(axis =1)
    I = np.where(L>1e-16)[0]
    return traj[I]

### In data_1 below:
    (1) there is no stationary points
    (2) there is no trajectories with more than 200 waypoints

In [8]:
for i in range(len(data_1)):
    data_1[i] = np.array(list(map(remove_segments, data_1[i])), dtype='object')
    L = np.array([len(data_1[i][j]) for j in range(len(data_1[i]))])
    I = np.where((L > 1000))[0]
    data_1[i] = data_1[i][I]

I = np.where(np.array([len(data_1[i]) for i in range(len(data_1))]) > 0)[0]
data_1 = data_1[I]
print("len(data_1) =", len(data_1))
print("selected users: \n", I)

len(data_1) = 136
selected users: 
 [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  48  49  50  51  52  54  55  56
  58  59  62  63  64  65  66  67  68  69  70  71  73  74  75  77  78  81
  82  83  84  85  88  89  92  93  95  96 100 101 103 104 111 112 113 114
 115 116 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133
 135 137 140 141 142 143 144 145 147 148 149 151 152 153 155 156 158 159
 160 163 165 167 168 169 172 177 179 180]


# Partitioning trajectories to less than 20 minutes long

In [9]:
def partition(trajectory, threshold=20):
    '''threshold is in minutes'''
    trajectories = []
    a = 24 * 60 * sum(trajectory[:,2][1:] - trajectory[:,2][:-1])
    if a <= threshold:
        return np.array(trajectory.reshape(1, len(trajectory), 3))
    else: 
        i = 0
        while a > threshold:
            j = i + 0
            val = 0
            while val < threshold: 
                if i < len(trajectory) - 1:
                    temp = val + 0
                    val += 24 * 60 * (trajectory[:,2][1:][i] - trajectory[:,2][:-1][i])
                    i += 1
                else: 
                    break
            if len(trajectory[j:i-1]) > 0:
                trajectories.append(trajectory[j:i-1])
            a = a - val
        if len(trajectory[i:]) > 0:
            trajectories.append(trajectory[i:])
    return np.array(trajectories)

### data3 below is the array of trajectories having less than 20 minutes long

In [13]:
data3 = [0] * len(data_1)

for i in range(len(data_1)):
    data3[i] = []
    for j in range(len(data_1[i])):
        A = partition(data_1[i][j], threshold=20)
        for k in range(len(A)):
            data3[i].append(A[k])
    data3[i] = np.array(data3[i], dtype='object')
    
data3 = np.array(data3, dtype='object')

data3.shape, data3[0].shape, data3[0][0].shape

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:24: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


((136,), (812,), (116, 3))

In [15]:
data4 = data3 + 0

data4 is the users having between 100 and 200 trajectories and each has length between 10 and 200 trajectory

In [16]:
for i in range(len(data4)):
    A = np.array([len(data4[i][j]) for j in range(len(data4[i]))])
    I = np.where((A > 10) & (A < 200))[0]
    data4[i] = data4[i][I]
    
print(len(data4))
A = np.array([len(data4[i]) for i in range(len(data4))])
chosen_users = np.where((A > 100) & (A < 200))[0]
data4 = data4[chosen_users]

print("chosen users:", chosen_users)
print("len(data4) =", len(data4))
A = [len(data4[i]) for i in range(len(data4))]
print("length of preprocessed users in data4: \n", np.sort(A))

136
chosen users: [  1   9  12  13  15  16  18  25  32  33  39  42  43  48  59  65  99 112
 125 130]
len(data4) = 20
length of preprocessed users in data4: 
 [108 112 113 117 120 121 122 132 140 151 152 154 159 163 171 174 177 181
 193 197]


### data2 is the same as data4 but without time dimension

In [17]:
data2 = data4 + 0
for i in range(len(data2)):
    data2[i] = np.array([data4[i][j][:,:2] for j in range(len(data4[i]))], dtype='object')
len(data2)

20

# Classifiers

In [22]:
CC = [100, 100, 10]
number_estimators = [50, 50]


clf0 = [make_pipeline(LinearSVC(dual=False, C=CC[0], tol=1e-5, 
                               class_weight ='balanced', max_iter=1000)), 
        "SVM, LinearSVC, C = "+str(CC[0])]
clf1 = [make_pipeline(StandardScaler(), svm.SVC(C=CC[1], kernel='rbf', gamma='auto', max_iter=200000)),
        "Gaussian SVM, C="+str(CC[1])+", gamma=auto"]
clf2 = [make_pipeline(StandardScaler(), svm.SVC(C=CC[2], kernel='poly', degree=3, max_iter=400000)),
        "Poly kernel SVM, C="+str(CC[2])+", deg=auto"]
clf3 = [DecisionTreeClassifier(), "Decision Tree"]
clf4 = [RandomForestClassifier(n_estimators=number_estimators[0]), 
         "RandomForestClassifier, n="+str(number_estimators[0])]
clf5 = [KNeighborsClassifier(n_neighbors=5), "KNN"]
clf6 = [LogisticRegression(solver='newton-cg'), "Logistic Regression"]
clf7 = [Perceptron(tol=1e-5, random_state=0, validation_fraction=0.01, 
                               class_weight= "balanced"), "Perceptron"]

clf = [clf0, clf1, clf2, clf3, clf4, clf5, clf6, clf7]
classifs = [item[0] for item in clf]
keys = [item[1] for item in clf]

# Chosen pairs of users

In [30]:
pairs_final
#pairs_final = [[4, 12], [4, 16], [5, 12], [8, 10]]

[[4, 12], [4, 16], [5, 12], [8, 10]]

# Calculations needed for correlation matrix

## Classification of pairs_100 with $v_Q$ with random $Q$ in each iteration

In [64]:
v_Q_errors_100 = []
v_Q_stds_100 = []

for pair in pairs_100:
    print(colored(f"pair={pair}", 'green'))
    classif = binaryClassificationAverageMajority(data[pair[0]], data[pair[1]], Q_size=20,
                                            epoch=100, num_trials_maj=1, classifiers=clf, 
                                            version='unsigned', test_size=0.3)
    A = classif.classification_v_Q()
    print(A[0])
    v_Q_errors_100.append(np.round(A[2], decimals=4).tolist())
    v_Q_stds_100.append(np.round(A[3], decimals=4).tolist())
    print("===========================================================================")

print('v_Q_errors_100 = \n', v_Q_errors_100)
print('v_Q_stds_100 = \n', v_Q_stds_100)
print("===========================================================================")
print(colored(f'average errors: {list(np.round(np.mean(v_Q_errors_100, 0), decimals=4))}', 'blue'))
print(colored(f'average stds: {list(np.round(np.mean(v_Q_stds_100, 0), decimals=4))}', 'green'))

np.savetxt('Claculated test errors for correlation/Beijing/random_V_Q_errors.csv', v_Q_errors_100, 
           delimiter=',')
np.savetxt('Claculated test errors for correlation/Beijing/random_V_Q_stds.csv', v_Q_stds_100, 
           delimiter=',')

pair=[0, 14]
total time = 23.848691940307617
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.3334      0.3441     0.0545
2  Gaussian SVM, C = 100, gamma =auto       0.3436      0.3442     0.0162
3   Poly kernel SVM, C = 10, deg=auto       0.3436      0.3442     0.0162
4                       Decision Tree       0.0000      0.1281     0.0356
5        RandomForestClassifier, n=50       0.0012      0.1214     0.0323
6                                 KNN       0.0941      0.1252     0.0344
7                 Logistic Regression       0.3436      0.3412     0.0160
8                          Perceptron       0.4343      0.4265     0.0977
pair=[3, 16]
total time = 20.389543056488037
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.4565      0.4607     0.0929
2  Gaussian SVM, C = 100, gamma =auto       0.2551      0.2590     0.0437
3   Poly kernel SVM, C

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, 

total time = 27.02938175201416
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.3941      0.3990     0.0349
2  Gaussian SVM, C = 100, gamma =auto       0.3883      0.4187     0.0434
3   Poly kernel SVM, C = 10, deg=auto       0.4050      0.4102     0.0170
4                       Decision Tree       0.0000      0.1405     0.0332
5        RandomForestClassifier, n=50       0.0015      0.1313     0.0308
6                                 KNN       0.0883      0.1302     0.0312
7                 Logistic Regression       0.4322      0.4433     0.0057
8                          Perceptron       0.4976      0.5001     0.0597
pair=[1, 18]
total time = 25.596420764923096
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.3660      0.3717     0.0137
2  Gaussian SVM, C = 100, gamma =auto       0.0326      0.0336     0.0158
3   Poly kernel SVM, C = 10, deg=aut

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


total time = 24.85150122642517
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.2727      0.2823     0.0736
2  Gaussian SVM, C = 100, gamma =auto       0.0487      0.0720     0.0273
3   Poly kernel SVM, C = 10, deg=auto       0.2806      0.2956     0.0521
4                       Decision Tree       0.0000      0.0615     0.0291
5        RandomForestClassifier, n=50       0.0007      0.0546     0.0240
6                                 KNN       0.0634      0.0940     0.0332
7                 Logistic Regression       0.3302      0.3479     0.0540
8                          Perceptron       0.4311      0.4302     0.0893
pair=[9, 17]


...

total time = 22.59690284729004
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.2703      0.2815     0.0924
2  Gaussian SVM, C = 100, gamma =auto       0.0483      0.0723     0.0259
3   Poly kernel SVM, C = 10, deg=auto       0.2780      0.2968     0.0545
4                       Decision Tree       0.0000      0.0583     0.0265
5        RandomForestClassifier, n=50       0.0005      0.0543     0.0233
6                                 KNN       0.0624      0.0898     0.0285
7                 Logistic Regression       0.3331      0.3506     0.0545
8                          Perceptron       0.4167      0.4288     0.0963
pair=[2, 14]
total time = 21.19256329536438
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.3236      0.3249     0.0385
2  Gaussian SVM, C = 100, gamma =auto       0.2722      0.2782     0.0372
3   Poly kernel SVM, C = 10, deg=auto

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


total time = 23.025547981262207
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.3107      0.3270     0.0560
2  Gaussian SVM, C = 100, gamma =auto       0.0786      0.1042     0.0333
3   Poly kernel SVM, C = 10, deg=auto       0.2963      0.3084     0.1084
4                       Decision Tree       0.0000      0.0747     0.0254
5        RandomForestClassifier, n=50       0.0002      0.0651     0.0245
6                                 KNN       0.0560      0.0936     0.0324
7                 Logistic Regression       0.3154      0.3111     0.0514
8                          Perceptron       0.4531      0.4569     0.0923
pair=[4, 5]
total time = 23.65599274635315
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.3093      0.3190     0.0412
2  Gaussian SVM, C = 100, gamma =auto       0.2012      0.2310     0.0332
3   Poly kernel SVM, C = 10, deg=auto

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


total time = 23.376121759414673
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.2703      0.2849     0.0887
2  Gaussian SVM, C = 100, gamma =auto       0.0495      0.0746     0.0297
3   Poly kernel SVM, C = 10, deg=auto       0.2769      0.2918     0.0563
4                       Decision Tree       0.0000      0.0565     0.0281
5        RandomForestClassifier, n=50       0.0006      0.0537     0.0272
6                                 KNN       0.0618      0.0944     0.0300
7                 Logistic Regression       0.3373      0.3517     0.0568
8                          Perceptron       0.4394      0.4337     0.0974
pair=[11, 16]
total time = 26.11333990097046
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.1712      0.1731     0.0323
2  Gaussian SVM, C = 100, gamma =auto       0.3286      0.3266     0.0225
3   Poly kernel SVM, C = 10, deg=au

## Classification for pairs_100 with $v_Q^{\varsigma}$ with random $Q$ in each iteration

In [147]:
v_Q_sigma_errors_100 = []
v_Q_sigma_stds_100 = []

for pair in pairs_100:
    print(colored(f"pair={pair}", 'green'))
    classif = binaryClassificationAverageMajority(data[pair[0]], data[pair[1]], Q_size=20,
                                            epoch=100, num_trials_maj=1, classifiers=clf, 
                                            version='signed', sigma=10, test_size=0.3)
    A = classif.classification_v_Q()
    print(A[0])
    v_Q_sigma_errors_100.append(np.round(A[2], decimals=4).tolist())
    v_Q_sigma_stds_100.append(np.round(A[3], decimals=4).tolist())
    print("===========================================================================")

print('v_Q_sigma_errors_100 = \n', v_Q_sigma_errors_100)
print('v_Q_sigma_stds_100 = \n', v_Q_sigma_stds_100)
print("===========================================================================")
print(colored(f'average errors: {list(np.round(np.mean(v_Q_sigma_errors_100, 0), decimals=4))}', 'blue'))
print(colored(f'average stds: {list(np.round(np.mean(v_Q_sigma_stds_100, 0), decimals=4))}', 'green'))

np.savetxt('Claculated test errors for correlation/Beijing/random_V_Q_sigma_errors.csv', 
           v_Q_sigma_errors_100, delimiter=',')
np.savetxt('Claculated test errors for correlation/Beijing/random_V_Q_sigma_stds.csv', 
           v_Q_sigma_stds_100, delimiter=',')

pair=[0, 14]


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, 

total time = 54.27748489379883
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.4494      0.4972     0.0568
2  Gaussian SVM, C = 100, gamma =auto       0.3333      0.3536     0.0181
3   Poly kernel SVM, C = 10, deg=auto       0.3712      0.3762     0.0278
4                       Decision Tree       0.0000      0.2290     0.0427
5        RandomForestClassifier, n=50       0.0014      0.2086     0.0450
6                                 KNN       0.1757      0.2505     0.0508
7                 Logistic Regression       0.3874      0.3907     0.0087
8                          Perceptron       0.4619      0.4616     0.1105
pair=[3, 16]


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, 

total time = 31.811262130737305
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.4384      0.5104     0.0590
2  Gaussian SVM, C = 100, gamma =auto       0.3743      0.4524     0.0473
3   Poly kernel SVM, C = 10, deg=auto       0.4635      0.4761     0.0114
4                       Decision Tree       0.0000      0.2746     0.0480
5        RandomForestClassifier, n=50       0.0014      0.2761     0.0478
6                                 KNN       0.2013      0.3197     0.0466
7                 Logistic Regression       0.4725      0.5017     0.0487
8                          Perceptron       0.4904      0.4977     0.0396
pair=[0, 11]


...

total time = 36.242223024368286
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.4667      0.5230     0.0433
2  Gaussian SVM, C = 100, gamma =auto       0.1835      0.3683     0.0449
3   Poly kernel SVM, C = 10, deg=auto       0.3873      0.4167     0.0307
4                       Decision Tree       0.0000      0.3140     0.0467
5        RandomForestClassifier, n=50       0.0003      0.2661     0.0460
6                                 KNN       0.2356      0.3591     0.0447
7                 Logistic Regression       0.4780      0.4719     0.0000
8                          Perceptron       0.5138      0.5191     0.0219
pair=[0, 3]


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, 

total time = 47.055508852005005
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.4416      0.4998     0.0765
2  Gaussian SVM, C = 100, gamma =auto       0.2319      0.2359     0.0295
3   Poly kernel SVM, C = 10, deg=auto       0.3931      0.3950     0.0396
4                       Decision Tree       0.0000      0.1852     0.0373
5        RandomForestClassifier, n=50       0.0007      0.1740     0.0372
6                                 KNN       0.1322      0.1948     0.0360
7                 Logistic Regression       0.3901      0.3926     0.0064
8                          Perceptron       0.5677      0.5734     0.1099
pair=[8, 19]
total time = 28.15678882598877
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.4646      0.4871     0.0513
2  Gaussian SVM, C = 100, gamma =auto       0.2956      0.2975     0.0372
3   Poly kernel SVM, C = 10, deg=aut

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)
/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


total time = 34.964287996292114
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.4256      0.4510     0.0504
2  Gaussian SVM, C = 100, gamma =auto       0.1830      0.2244     0.0410
3   Poly kernel SVM, C = 10, deg=auto       0.3905      0.4031     0.0055
4                       Decision Tree       0.0000      0.1797     0.0361
5        RandomForestClassifier, n=50       0.0009      0.1629     0.0340
6                                 KNN       0.1410      0.2146     0.0431
7                 Logistic Regression       0.3971      0.4000     0.0000
8                          Perceptron       0.5662      0.5658     0.0747
v_Q_sigma_errors_100 = 
 [[0.4972, 0.3536, 0.3762, 0.229, 0.2086, 0.2505, 0.3907, 0.4616], [0.5104, 0.4524, 0.4761, 0.2746, 0.2761, 0.3197, 0.5017, 0.4977], [0.5164, 0.2773, 0.4062, 0.2298, 0.214, 0.2475, 0.5085, 0.5119], [0.5266, 0.224, 0.3922, 0.1216, 0.1143, 0.1254, 0.3792, 0.5309], [0.5311, 0.1744

## Classification for pairs_100 with endpoints

In [149]:
endpoints_errors_100 = []
endpoints_stds_100 = []
i = 0

for pair in pairs_100:
    print("i=", i)
    print(colored(f"pair={pair}", 'green'))
    classif = binaryClassificationAverageMajority(data[pair[0]], data[pair[1]], Q_size=20,
                                            epoch=100, num_trials_maj=1, classifiers=clf, 
                                            version='unsigned', test_size=0.3)
    A = classif.endpoint_classification()
    print(A[0])
    endpoints_errors_100.append(np.round(A[2], decimals=4).tolist())
    endpoints_stds_100.append(np.round(A[3], decimals=4).tolist())
    print("===========================================================================")
    i += 1

print('endpoints_errors_100 = \n', endpoints_errors_100)
print('endpoints_stds_100 = \n', endpoints_stds_100)
print("===========================================================================")
print(colored(f'average errors: {list(np.round(np.mean(endpoints_errors_100, 0), decimals=4))}', 'blue'))
print(colored(f'average stds: {list(np.round(np.mean(endpoints_stds_100, 0), decimals=4))}', 'green'))

np.savetxt('Claculated test errors for correlation/Beijing/endpoints_errors.csv', endpoints_errors_100, 
           delimiter=',')
np.savetxt('Claculated test errors for correlation/Beijing/endpoints_stds.csv', endpoints_stds_100, 
           delimiter=',')

i= 0
pair=[0, 14]
total time = 9.33417296409607
Number of trials = 100
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.3613      0.3602     0.0476
2  Gaussian SVM, C = 100, gamma =auto       0.2099      0.2335     0.0406
3   Poly kernel SVM, C = 10, deg=auto       0.3270      0.3414     0.0234
4                       Decision Tree       0.0000      0.0879     0.0317
5        RandomForestClassifier, n=50       0.0001      0.0637     0.0253
6                                 KNN       0.0572      0.0901     0.0299
7                 Logistic Regression       0.3441      0.3398     0.0195
8                          Perceptron       0.4845      0.4841     0.1051
i= 1
pair=[3, 16]


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


total time = 9.611308097839355
Number of trials = 100
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.4436      0.4687     0.0614
2  Gaussian SVM, C = 100, gamma =auto       0.2475      0.2508     0.0382
3   Poly kernel SVM, C = 10, deg=auto       0.3785      0.3793     0.0300
4                       Decision Tree       0.0000      0.0737     0.0304
5        RandomForestClassifier, n=50       0.0004      0.0535     0.0285
6                                 KNN       0.0707      0.1131     0.0357
7                 Logistic Regression       0.3666      0.3976     0.1058
8                          Perceptron       0.4851      0.4858     0.0412
i= 2
pair=[0, 11]
total time = 10.86427617073059
Number of trials = 100
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.1608      0.1656     0.0356
2  Gaussian SVM, C = 100, gamma =auto       0.1002      0.11

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/_base.py:258: ConvergenceWarning: Solver terminated early (max_iter=400000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  % self.max_iter, ConvergenceWarning)


total time = 9.23293685913086
Number of trials = 100
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.3749      0.3946     0.0534
2  Gaussian SVM, C = 100, gamma =auto       0.0220      0.0491     0.0198
3   Poly kernel SVM, C = 10, deg=auto       0.2184      0.2332     0.0711
4                       Decision Tree       0.0000      0.0493     0.0244
5        RandomForestClassifier, n=50       0.0003      0.0389     0.0214
6                                 KNN       0.0416      0.0626     0.0274
7                 Logistic Regression       0.4265      0.4313     0.0412
8                          Perceptron       0.4355      0.4390     0.0000
i= 10
pair=[9, 17]
total time = 8.565035104751587
Number of trials = 100
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.1691      0.1726     0.0264
2  Gaussian SVM, C = 100, gamma =auto       0.1721      0.17

...

total time = 8.92532467842102
Number of trials = 100
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.4025      0.4114     0.0461
2  Gaussian SVM, C = 100, gamma =auto       0.1661      0.2118     0.0295
3   Poly kernel SVM, C = 10, deg=auto       0.4151      0.4365     0.0199
4                       Decision Tree       0.0000      0.1161     0.0268
5        RandomForestClassifier, n=50       0.0002      0.0792     0.0235
6                                 KNN       0.0714      0.1061     0.0271
7                 Logistic Regression       0.4552      0.4595     0.0178
8                          Perceptron       0.4761      0.4773     0.0110
i= 43
pair=[3, 7]
total time = 7.761270046234131
Number of trials = 100
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.4081      0.4146     0.0357
2  Gaussian SVM, C = 100, gamma =auto       0.1558      0.170

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


total time = 7.871549129486084
Number of trials = 100
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.2441      0.2517     0.0281
2  Gaussian SVM, C = 100, gamma =auto       0.2129      0.2152     0.0320
3   Poly kernel SVM, C = 10, deg=auto       0.2839      0.2908     0.0466
4                       Decision Tree       0.0000      0.0596     0.0201
5        RandomForestClassifier, n=50       0.0003      0.0351     0.0204
6                                 KNN       0.0341      0.0422     0.0193
7                 Logistic Regression       0.2236      0.2267     0.0327
8                          Perceptron       0.5468      0.5444     0.0342
i= 58
pair=[15, 17]
total time = 7.867694139480591
Number of trials = 100
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.1685      0.1733     0.0291
2  Gaussian SVM, C = 100, gamma =auto       0.1701      0.

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


total time = 8.583748817443848
Number of trials = 100
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.2507      0.2643     0.0340
2  Gaussian SVM, C = 100, gamma =auto       0.2210      0.2266     0.0278
3   Poly kernel SVM, C = 10, deg=auto       0.2969      0.3074     0.0446
4                       Decision Tree       0.0000      0.0473     0.0270
5        RandomForestClassifier, n=50       0.0003      0.0364     0.0229
6                                 KNN       0.0327      0.0488     0.0262
7                 Logistic Regression       0.2396      0.2480     0.0290
8                          Perceptron       0.4944      0.4944     0.0647
i= 76
pair=[0, 5]
total time = 9.812248945236206
Number of trials = 100
                           Classifier  Train Error  Test Error  Std Error
1             SVM, LinearSVC, C = 100       0.3546      0.3609     0.0377
2  Gaussian SVM, C = 100, gamma =auto       0.1188      0.15

# KNN for correlation matrix with 100 pairs

In [68]:
import numpy as np
import time
import pandas as pd
import random
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import traj_dist.distance as tdist
import pickle
import tslearn
from tslearn.metrics import dtw as dtw_tslearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from trjtrypy.distances import d_Q_pi
from termcolor import colored

from KNN_Class_Colab import KNN

metrics = ['discret_frechet', 'hausdorff', 'dtw', 'sspd', 'erp', 'edr', 'lcss', 
           fastdtw, dtw_tslearn, 'd_Q_pi']

path = 'Calculated Distance Matrices for KNN-Beijing-Correlation/'

train_test_mean_median_std_KNN_errors = []
test_KNN_errors = []
std_KNN_errors = []

for pair in pairs_100:
    print('pair =', pair)
    temp = []
    for i in range(len(metrics)):
        KNN_class = KNN(data[pair[0]], data[pair[1]], metric=metrics[i], gamma=1e-10, 
                        eps_edr=0.01, eps_lcss=0.01, Q_size=20, Q=None, p=2, 
                        path=path+str(metrics[i])+'-'+str(pair), 
                        test_size=0.3, n_neighbors=5, num_trials=100, pair=[0,1])

        KNN_class.write_matrix_to_csv()
        A = KNN_class.KNN_average_error()
        temp.append(A[1:])        
        print(i, A[0])
        print("=======================================================================")
    
    train_test_mean_median_std_KNN_errors.append(temp)
    test_KNN_errors.append(np.array(temp)[:,1])
    std_KNN_errors.append(np.array(temp)[:,3])
    print(colored("******************************************************************************", 'red'))
    print(colored("******************************************************************************", 'red'))
    print(colored("******************************************************************************", 'red'))
    
train_test_mean_median_std_KNN_errors = np.array(train_test_mean_median_std_KNN_errors)
test_KNN_errors = np.array(test_KNN_errors)
std_KNN_errors = np.array(std_KNN_errors)

list_test_error = list(np.round(np.mean(test_KNN_errors, 0), decimals=4))
list_std_error = list(np.round(np.mean(std_KNN_errors, 0), decimals=4))
print(colored(f'average test errors of 100 pairs: \n {list_test_error}', 'magenta'))
print(colored(f'average stds of 100 pairs: \n {list_std_error}', 'yellow'))


path = 'Claculated test errors for correlation/Beijing/KNN_test_errors_Beijing.csv'
np.savetxt(path, test_KNN_errors, delimiter=',')

pair = [0, 14]
num_trials = 100
total time = 3.1312477588653564
0                           Classifier  Train Error   Mean  Median     Std
1  KNN; discret_frechet; pair [0, 1]       0.0701  0.098   0.093  0.0262
num_trials = 100
total time = 3.6010968685150146
1                     Classifier  Train Error    Mean  Median     Std
1  KNN; hausdorff; pair [0, 1]       0.0702  0.0901   0.093  0.0281
num_trials = 100
total time = 3.6492059230804443
2               Classifier  Train Error    Mean  Median     Std
1  KNN; dtw; pair [0, 1]       0.0486  0.0736  0.0698  0.0304
num_trials = 100
total time = 3.3584039211273193
3                Classifier  Train Error    Mean  Median     Std
1  KNN; sspd; pair [0, 1]        0.047  0.0637  0.0581  0.0281


/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/traj_dist-1.1-py3.7-macosx-10.9-x86_64.egg/traj_dist/distance.py:552: UserWarning: g parameter should be specified for metric erp. Default is 
  warnings.warn("g parameter should be specified for metric erp. Default is ")


num_trials = 100
total time = 4.132692813873291
4               Classifier  Train Error    Mean  Median     Std
1  KNN; erp; pair [0, 1]       0.2809  0.4249  0.4186  0.0452
num_trials = 100
total time = 3.6947038173675537
5               Classifier  Train Error    Mean  Median     Std
1  KNN; edr; pair [0, 1]       0.1271  0.1652  0.1628  0.0428
num_trials = 100
total time = 3.8618202209472656
6                Classifier  Train Error    Mean  Median     Std
1  KNN; lcss; pair [0, 1]       0.1173  0.1345  0.1279  0.0384
num_trials = 100
total time = 3.3383429050445557
7                                       Classifier  Train Error    Mean  Median  \
1  KNN; <built-in function fastdtw>; pair [0, 1]         0.05  0.0786  0.0814   

      Std  
1  0.0276  
num_trials = 100
total time = 4.234208822250366
8                                           Classifier  Train Error    Mean  \
1  KNN; <function dtw at 0x7fc2aa83cd40>; pair [0...       0.0561  0.0848   

   Median     Std  
1  0.0814  

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/traj_dist-1.1-py3.7-macosx-10.9-x86_64.egg/traj_dist/distance.py:552: UserWarning: g parameter should be specified for metric erp. Default is 
  warnings.warn("g parameter should be specified for metric erp. Default is ")


num_trials = 100
total time = 3.2703707218170166
4               Classifier  Train Error   Mean  Median     Std
1  KNN; erp; pair [0, 1]       0.2694  0.458  0.4507  0.0528
num_trials = 100
total time = 2.5313501358032227
5               Classifier  Train Error    Mean  Median   Std
1  KNN; edr; pair [0, 1]       0.2274  0.2731  0.2817  0.04
num_trials = 100
total time = 2.7694759368896484
6                Classifier  Train Error    Mean  Median     Std
1  KNN; lcss; pair [0, 1]       0.1274  0.1862  0.1831  0.0437
num_trials = 100
total time = 2.747176170349121
7                                       Classifier  Train Error    Mean  Median  \
1  KNN; <built-in function fastdtw>; pair [0, 1]       0.0568  0.0963  0.0986   

      Std  
1  0.0312  
num_trials = 100
total time = 2.9679648876190186
8                                           Classifier  Train Error    Mean  \
1  KNN; <function dtw at 0x7fc2aa83cd40>; pair [0...       0.0583  0.0996   

   Median     Std  
1  0.0986  0.033

...

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/traj_dist-1.1-py3.7-macosx-10.9-x86_64.egg/traj_dist/distance.py:552: UserWarning: g parameter should be specified for metric erp. Default is 
  warnings.warn("g parameter should be specified for metric erp. Default is ")


num_trials = 100
total time = 2.4971487522125244
4               Classifier  Train Error    Mean  Median     Std
1  KNN; erp; pair [0, 1]       0.2475  0.3649   0.375  0.0416
num_trials = 100
total time = 2.358511209487915
5               Classifier  Train Error    Mean  Median     Std
1  KNN; edr; pair [0, 1]       0.0278  0.0594  0.0417  0.0422
num_trials = 100
total time = 2.4706308841705322
6                Classifier  Train Error    Mean  Median    Std
1  KNN; lcss; pair [0, 1]       0.0302  0.0596  0.0556  0.033
num_trials = 100
total time = 2.438431978225708
7                                       Classifier  Train Error  Mean  Median  \
1  KNN; <built-in function fastdtw>; pair [0, 1]       0.0267  0.03  0.0278   

      Std  
1  0.0231  
num_trials = 100
total time = 2.440922975540161
8                                           Classifier  Train Error    Mean  \
1  KNN; <function dtw at 0x7fc2aa83cd40>; pair [0...       0.0239  0.0257   

   Median     Std  
1  0.0278  0.0158 

/Users/hasan/opt/anaconda3/lib/python3.7/site-packages/traj_dist-1.1-py3.7-macosx-10.9-x86_64.egg/traj_dist/distance.py:552: UserWarning: g parameter should be specified for metric erp. Default is 
  warnings.warn("g parameter should be specified for metric erp. Default is ")


num_trials = 100
total time = 3.358842134475708
4               Classifier  Train Error    Mean  Median     Std
1  KNN; erp; pair [0, 1]       0.2797  0.4283  0.4222  0.0451
num_trials = 100
total time = 3.1744210720062256
5               Classifier  Train Error    Mean  Median     Std
1  KNN; edr; pair [0, 1]       0.0558  0.0748  0.0667  0.0275
num_trials = 100
total time = 3.31966495513916
6                Classifier  Train Error    Mean  Median    Std
1  KNN; lcss; pair [0, 1]         0.06  0.0758  0.0778  0.027
num_trials = 100
total time = 3.1685099601745605
7                                       Classifier  Train Error    Mean  Median  \
1  KNN; <built-in function fastdtw>; pair [0, 1]       0.0283  0.0426  0.0444   

      Std  
1  0.0227  
num_trials = 100
total time = 3.4116711616516113
8                                           Classifier  Train Error    Mean  \
1  KNN; <function dtw at 0x7fc2aa83cd40>; pair [0...       0.0368  0.0539   

   Median     Std  
1  0.0556  0.0

NameError: name 'KNN_test_errors_Beijing' is not defined

In [77]:
for i in range(100):
    path = 'Claculated test errors for correlation/Beijing/train_test_mean_median_std_KNN_errors/'+str(i)+'.csv'
    np.savetxt(path, train_test_mean_median_std_KNN_errors[i], delimiter=',')

In [122]:
import numpy as np
import time
import pandas as pd
import random
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw
import traj_dist.distance as tdist
import pickle
import tslearn
from tslearn.metrics import dtw as dtw_tslearn
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from trjtrypy.distances import d_Q_pi
from termcolor import colored
from sdtw import SoftDTW
from sdtw.distance import SquaredEuclidean

from KNN_Class import KNN

path = 'Calculated Distance Matrices for KNN-Beijing-Correlation/'

train_test_mean_median_std_KNN_soft_dtw_errors = []
test_KNN_soft_dtw_errors = []
std_KNN_soft_dtw_errors = []
i = 0
for pair in pairs_100:
    st = time.time()
    print('pair =', pair)
    KNN_class = KNN(data[pair[0]], data[pair[1]], metric=SoftDTW, gamma=1e-15, 
                    eps_edr=0.01, eps_lcss=0.01, Q=None, p=2, 
                    path=path+'soft-dtw'+'-'+str(pair), 
                    test_size=0.3, n_neighbors=5, num_trials=100, pair=[0,1])

    KNN_class.write_matrix_to_csv()
    A = KNN_class.KNN_average_error()
    train_test_mean_median_std_KNN_soft_dtw_errors.append(A[1:])        
    print(i, A[0])
    i += 1
    print(colored(f"total time for pair {i}: {time.time()-st}", 'red'))
    print("=======================================================================")
    
train_test_mean_median_std_KNN_soft_dtw_errors = np.array(train_test_mean_median_std_KNN_soft_dtw_errors)
test_KNN_soft_dtw_errors = train_test_mean_median_std_KNN_soft_dtw_errors[:,1]
std_KNN_soft_dtw_errors = train_test_mean_median_std_KNN_soft_dtw_errors[:,3]

print(np.mean(test_KNN_soft_dtw_errors))

test_error = np.round(np.mean(test_KNN_soft_dtw_errors, 0), decimals=4)
std_error = np.round(np.mean(std_KNN_soft_dtw_errors, 0), decimals=4)
print(colored(f'average test errors of 100 pairs: {test_error}', 'magenta'))
print(colored(f'average stds of 100 pairs: {std_error}', 'yellow'))

path = 'Claculated test errors for correlation/Beijing/KNN_soft_dtw_test_errors_Beijing.csv'
np.savetxt(path, test_KNN_soft_dtw_errors, delimiter=',')


pair = [0, 14]
num_trials = 100
total time = 3.916410207748413
0                                           Classifier  Train Error   Mean  \
1  KNN; <class 'sdtw.soft_dtw.SoftDTW'>; pair [0, 1]       0.0552  0.085   

   Median     Std  
1  0.0814  0.0265  
total time for pair 1: 32.770118951797485
pair = [3, 16]
num_trials = 100
total time = 2.3709142208099365
1                                           Classifier  Train Error    Mean  \
1  KNN; <class 'sdtw.soft_dtw.SoftDTW'>; pair [0, 1]       0.0578  0.0989   

   Median     Std  
1  0.0986  0.0296  
total time for pair 2: 19.322986125946045
pair = [0, 11]
num_trials = 100
total time = 4.947636365890503
2                                           Classifier  Train Error   Mean  \
1  KNN; <class 'sdtw.soft_dtw.SoftDTW'>; pair [0, 1]       0.0293  0.042   

   Median     Std  
1  0.0377  0.0221  
total time for pair 3: 49.43159604072571
pair = [1, 3]
num_trials = 100
total time = 3.9852328300476074
3                                  

In [124]:
path = 'Claculated test errors for correlation/Beijing/KNN_test_errors_Beijing.csv'
A = np.array(pd.read_csv(path, header=None))
path = 'Claculated test errors for correlation/Beijing/KNN_soft_dtw_test_errors_Beijing.csv'
B = np.array(pd.read_csv(path, header=None))
C = np.concatenate((A, B), 1)
print(C.shape)

(100, 11)


In [125]:
path = 'Claculated test errors for correlation/Beijing/train_test_mean_median_std_KNN_errors_11_distances.csv'
np.savetxt(path, C, delimiter=',')


In [ ]:
path = 'Claculated test errors for correlation/Beijing/train_test_mean_median_std_KNN_errors_11_distances.csv'
KNN_test_errors = np.array(pd.read_csv(path, header=None)).T

path = 'Claculated test errors for correlation/Beijing/KNN_test_errors_Beijing.csv'
np.savetxt(path, KNN_test_errors, delimiter=',')

## KNN with LSH

In [110]:
from KNN_with_LSH_class import KNN_with_LSH

In [75]:
train_test_mean_median_std_KNN_LSH_errors = []
test_KNN_LSH_errors = []
std_KNN_LSH_errors = []
i = 0
for pair in pairs_100:
    st = time.time()
    print('pair =', pair)
    KNN_with_LSH_class = KNN_with_LSH(data[pair[0]], data[pair[1]], number_circles=20, 
                                      num_trials=100)
    A = KNN_with_LSH_class.KNN_LSH_average_error()
    train_test_mean_median_std_KNN_LSH_errors.append(A[1:])        
    print(i, A[0])
    i += 1
    print(colored(f"total time for pair {i}: {time.time()-st}", 'red'))
    print("=======================================================================")
    
train_test_mean_median_std_KNN_LSH_errors = np.array(train_test_mean_median_std_KNN_LSH_errors)
test_KNN_LSH_errors = train_test_mean_median_std_KNN_LSH_errors[:,1]
std_KNN_LSH_errors = train_test_mean_median_std_KNN_LSH_errors[:,3]

print(np.mean(test_KNN_LSH_errors))

test_error = np.round(np.mean(test_KNN_LSH_errors, 0), decimals=4)
std_error = np.round(np.mean(std_KNN_LSH_errors, 0), decimals=4)
print(colored(f'average test errors of 100 pairs: {test_error}', 'magenta'))
print(colored(f'average stds of 100 pairs: {std_error}', 'yellow'))

path = 'Claculated test errors for correlation/Beijing/KNN_LSH_test_errors_Beijing.csv'
np.savetxt(path, test_KNN_LSH_errors, delimiter=',')

path = 'Claculated test errors for correlation/Beijing/train_test_mean_median_std_KNN_LSH_errors.csv'
np.savetxt(path, train_test_mean_median_std_KNN_LSH_errors, delimiter=',')


pair = [0, 14]
num_trials = 100
total time = 15.31488299369812
0      Classifier  Train Error  Test Error     std
1  KNN with LSH       0.4614      0.4626  0.0865
total time for pair 1: 15.317207098007202
pair = [3, 16]
num_trials = 100
total time = 10.830833911895752
1      Classifier  Train Error  Test Error    std
1  KNN with LSH       0.4978      0.4975  0.021
total time for pair 2: 10.833698987960815
pair = [0, 11]
num_trials = 100
total time = 18.772157907485962
2      Classifier  Train Error  Test Error     std
1  KNN with LSH       0.3293       0.331  0.0461
total time for pair 3: 18.775162935256958
pair = [1, 3]
num_trials = 100
total time = 13.95433497428894
3      Classifier  Train Error  Test Error     std
1  KNN with LSH       0.4473      0.4457  0.1048
total time for pair 4: 13.956171035766602
pair = [1, 5]
num_trials = 100
total time = 16.632582187652588
4      Classifier  Train Error  Test Error     std
1  KNN with LSH       0.4279      0.4252  0.1086
total time for pai

In [88]:
path = 'Claculated test errors for correlation/Beijing/KNN_test_errors_Beijing.csv'
A = np.array(pd.read_csv(path, header=None)).T

path = 'Claculated test errors for correlation/Beijing/KNN_LSH_test_errors_Beijing.csv'
B = np.array(pd.read_csv(path, header=None))

D = np.concatenate((A, B), 1)
print(D.shape)

(100, 12)


In [90]:
path = 'Claculated test errors for correlation/Beijing/train_test_mean_median_std_KNN_errors_12_distances.csv'
np.savetxt(path, D, delimiter=',')


In [93]:
path = 'Claculated test errors for correlation/Beijing/train_test_mean_median_std_KNN_errors_12_distances.csv'
KNN_test_errors = np.array(pd.read_csv(path, header=None)).T

path = 'Claculated test errors for correlation/Beijing/KNN_test_errors_Beijing.csv'
np.savetxt(path, KNN_test_errors, delimiter=',')